In [17]:
import os.path
from PIL import Image, ImageDraw, ImageFont
import requests
import glob
import xml.etree.cElementTree as ele
from io import BytesIO

#For Pretty_Print in XML
#for indenting
def dent(element, level=0):
  i = "\n" + level*"  "
  if len(element):
    if not element.text or not element.text.strip():
      element.text = i + "  "
    if not element.tail or not element.tail.strip():
      element.tail = i
    for element in element:
      dent(element, level+1)
    if not element.tail or not element.tail.strip():
      element.tail = i
  else:
    if level and (not element.tail or not element.tail.strip()):
      element.tail = i

#Getting addresses
cwd = os.getcwd()
path1 = os.path.join(cwd,'NFPA dataset/*.jpg')
path2 = os.path.join(cwd,'NFPA dataset/*.txt')
images=glob.glob(path1)
files = glob.glob(path2)

#Output File Paths
pathforimages = os.path.join(cwd,'OutputImages\\')
pathforlabels = os.path.join(cwd,'OutputLabels\\')

if not os.path.exists(pathforimages):
    os.makedirs(pathforimages)
if not os.path.exists(pathforlabels):
    os.makedirs(pathforlabels)

for image,name in zip(images,files):
    img = Image.open(image)
    draw = ImageDraw.Draw(img)
    width, height = img.size

    (fpath, fname) = os.path.split(image)

    # Assuming these codes for depths
    mode_to_depth = {'L': 1, 'P': 2, 'RGB': 3, 'RGBA': 4, 'CMYK': 5, 'YCbCr': 6, 'I': 7, 'F': 8}
    depth = mode_to_depth[img.mode]

    # XML File
    root = ele.Element("annotation")
    folder = ele.SubElement(root, "folder").text = "NFPA dataset"
    filename = ele.SubElement(root, "filename").text = fname
    path = ele.SubElement(root, "path").text = image
    source = ele.SubElement(root, "source")
    database = ele.SubElement(source, "database").text = "Unknown"

    size = ele.SubElement(root, "size")
    width1 = ele.SubElement(size, "width").text = str(width)
    height1 = ele.SubElement(size, "height").text = str(height)
    depth1 = ele.SubElement(size, "depth").text = str(depth)

    width = float(width1)
    height = float(height1)

    with open(name) as f:
        for line in f:
            split = line.split()
            n,x,y,w,h = split
            x = float(x)
            y = float(y)
            w = float(w)
            h = float(h)

            x = x*width
            w = w*width
            y = y*height
            h = h*height

            a = float(x-w/2)
            b = float(y-h/2)
            c = float(x+w/2)
            d = float(y+h/2)
            
            #a=xmin, b=ymin ,c=xmax, d=ymax
            draw.rectangle(((a,b), (c,d)), outline="red")

            #XML Continued
            object = ele.SubElement(root, "object")
            name = ele.SubElement(object, "name").text = "nfpa"
            pose = ele.SubElement(object, "pose").text = "Unspecified"
            truncated = ele.SubElement(object, "truncated").text = "0"
            difficult = ele.SubElement(object, "difficult").text = "0"
            bndbox = ele.SubElement(object, "bndbox")
            xmin = ele.SubElement(bndbox, "xmin").text = str(a)
            ymin = ele.SubElement(bndbox, "ymin").text = str(b)
            xmax = ele.SubElement(bndbox, "xmax").text = str(c)
            ymax = ele.SubElement(bndbox, "ymax").text = str(d)

    img.save(newpath1 + fname)
    dent(root)
    tree = ele.ElementTree(root)
    (fn, ext) = os.path.splitext(fname)
    tree.write(newpath2 + fn + ".xml")

In [18]:
newpath1


'C:\\Users\\vipul\\OutputImages\\'